# SSH Tunnel Setup .
In order to run *ssh* into the Google Colab Virtual Machine (VM), we need to: 
*   Start the ssh daemon on the VM
*   Reverse proxy **ngrok** 






## Installation 
Install the python wrapper to interface with ngrok.<br>
The colab-ssh is a light-weight library that enables you to connect to a Google Colab VM using an SSH tunnel.<p>

**Note**: colab-ssh is easiest way to install the ssh daemon and run the 
daemon. However, for finer control other methods are prefered.

In [9]:
# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

Requirement already up-to-date: colab_ssh in /usr/local/lib/python3.6/dist-packages (0.2.64)


## Setup

To setup the reverse proxy we need ngrok authentication token. The token can be obtained from https://dashboard.ngrok.com/auth/your-authtoken. You may have to register if you don't have an account.<p>

Note down the secret key that you input to use it while tunnelling through ssh.<br> 
Note down the Hostname, User, and Port for reverse proxy.  

In [10]:
from colab_ssh import launch_ssh, init_git
ngrokToken = '1k6o811Gdt5dhRFYTxzdJGNdWqM_6HRtaxQMH9cjwasr4uJni'

from getpass import getpass
secret = getpass('Enter the secret key: ')

launch_ssh(ngrokToken, secret)

Enter the secret key: ··········
Successfully running 0.tcp.ngrok.io:19027
[Optional] You can also connect with VSCode SSH Remote extension using this configuration:

	Host google_colab_ssh
		HostName 0.tcp.ngrok.io
		User root
		Port 19027
	  


## Make sure SSH daemon is up and running

Check the status of the SSH service. Start the service if not running.

In [42]:
import os
import subprocess

def is_service_running(name):
  with open(os.devnull, 'wb') as hide_output:
    exit_code = subprocess.Popen(['service', name, 'status'],
                                 stdout=hide_output, 
                                 stderr=hide_output).wait()
    return exit_code == 0

if is_service_running('ssh'):
  print ('sshd is running')
else:
  print ('sshd is not running. Starting ...')
  os.system('service ssh start')
  if is_service_running('ssh'):
    print ('sshd started')
  


sshd is running


In [44]:
#optional step for debugging
!ps -ef | grep "sshd"

root        2844       1  0 16:06 ?        00:00:00 /usr/sbin/sshd
root        2886      56  0 16:08 ?        00:00:00 /bin/bash -c ps -ef | grep "sshd"
root        2888    2886  0 16:08 ?        00:00:00 /bin/bash -c ps -ef | grep "sshd"


## Instruction to SSH from a Terminal

From a terminal run the following command


> ssh \<User>@\<HostName> -p \<Port> <br>
> Ex: *ssh root@2.tcp.ngrok.io -p 17781*

When prompted for the **password** enter the **secret key**.

### Accessing Google Drive from SSH session

Mount the google drive from this Notebook. The mount point will be visibile within SSH session.



# Make sure that GPU is attached.
Execute the following cell.<br>
When the GPU is not attached the message(output) for this cell will indicate that the "NVIDIA-SMI" failed. If the GPU is not attached, set the Hardware Accelerator to GPU from colab menu:<br>
> *Runtime -> Change runtime type -> Hardware Accelerator*.    

Re-run the cell if you happpened to set up the Hardware Accelerator to make sure the the GPU is attached. 

In [3]:
!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Sun Nov 15 15:29:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Debugging

In [ ]:
!which nvidia-smi
!echo $PATH
#!service ssh stop
!which nvcc

/usr/bin/nvidia-smi
/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin
/usr/local/cuda/bin/nvcc


In [ ]:
!whoami

root


# Run CUDA from Notebook

Run the command to install a small extension to run nvcc from the Notebook cells.

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-pvxcki_g
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-pvxcki_g
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=fe791f85de40778123bad70a088a797aa16365261a67dd1a4c3ce8d8d106348a
  Stored in directory: /tmp/pip-ephem-wheel-cache-zuqk0230/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


Load the extension using the code given below

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu 
#include <iostream> 

int main() 
{ 
 printf("Welcome to the CUDA world!");
 return 0;
} 


Welcome to the CUDA world!


# Additional Helpful Settings

## Mounting Google Drive 

The following cell(s) will mount google drive to the virtual machine's file system.<br>
It defines a **GDRIVE_HOME** variable which can be used later 

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
GDRIVE_HOME = "/content/drive/My\ Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls -la $GDRIVE_HOME/Colab\ Notebooks

total 243
-rw------- 1 root root  19537 Oct  6 00:31  1_Preliminaries.ipynb
-rw------- 1 root root  15985 Nov 11 01:51 'Copy of SSH_to_colab_and_more.ipynb'
-rw------- 1 root root   4456 Aug 24 22:55  CUDA.ipynb
-rw------- 1 root root   7126 Oct  5 15:50  data_loader.py
-rw------- 1 root root 153399 Aug 12 19:45  FederatedLearning.ipynb
drwx------ 2 root root   4096 Oct  5 16:27  __pycache__
-rw------- 1 root root  14715 Oct 15 18:48  scratch.ipynb
-rw------- 1 root root  12719 Nov 12 16:34  SSH_to_colab_and_more.ipynb
-rw------- 1 root root   3726 Sep 26 21:27  Untitled0.ipynb
-rw------- 1 root root   6835 Aug 17 18:52  Untitled.ipynb
-rw------- 1 root root   3505 Oct  6 00:15  vocabulary.py


# FAQ

In [46]:
#! export PATH=/usr/local/nvidia/bin:/usr/local/cuda/bin:$PATH

#apt-get install vim

Sample code:



```
#include<stdio.h>
#include<stdlib.h>

__global__ void print_from_gpu(void) {
    printf("Hello World! from thread [%d,%d] \
        From device\n", threadIdx.x,blockIdx.x);
}

int main(void) {
    printf("Hello World from host!\n");
    print_from_gpu<<<1,1>>>();
    cudaDeviceSynchronize();
    return 0;
}
```




Compilation instruction
---
$ nvcc -o hello hello.cu
---